In [60]:
import pandas as pd

from copy import deepcopy
from abc import abstractmethod

from numba import njit
from numba import int32, float32    # import the types
from numba.experimental import jitclass

spec = [
    ('value', int32),               # a simple scalar field
    ('array', float32[:]),          # an array field
]

# Master algo

In [62]:
@jitclass(spec)
class Master(object):
    """
    Master algorithm managing the collaboration between the local data sites
    """
    
    @njit
    def __init__(self, collabs=None, Xs=None, Ys=None, max_iter=0, *args, **kwargs):
        """
        Parameters:
        collabs: list(Collaborator)
            List of instances of type "Collaborator".
        X: list(pandas.DataFrame)
            List of dataframes with same size than collabs.
            The view of each collaborators, with acces to possibly different
            set of individuals (identified by the index column).
        Y (optional): list(pandas.DataFrame)
            List of dataframes, one for each collaborator. With the indices and the labels.
        max_iter (optional): int
            Maximum number of iterations in the collaborative step
            
        notes:
        02/02 14:16 - Xs and Ys are not necessary parameters, as we can get them from each collab.
                      Consider removing them.
        """
        if collabs:
            self.collabs, self.P = collabs, len(collabs)
            
        if X:
            self.X = X
            
        if Y:
            self.Y = Y
        
        self.max_iter = max_iter
        # create a log that will contain information about each step of the process
        self.log = []
        
    @njit
    def launch_collab(self):
        """
        Proceed to the collaboration.
        
        """
        
        # each collaborator fits its parameters in the local step
        to_log = []
        for collab in self.collabs:
            collab.local_step()
            to_log.append(deepcopy(collab.log_local()))
        # log
        self.log.append(deepcopy(to_log))
        
        n_iter=0
        while True:
            stop, to_log = True, []
            
            # each data site is in turn considered as the local data site
            for p, collab in collabs:
                # it is provided with every remote partition matrix
                remote_partitions = self.get_partitions_except_p(p)
                collab.collaborate(remote_partitions)
                to_log.append(deepcopy(collab.log_collab()))
                if not collab.stop_criterion():
                    stop = False
            
            # log the results
            self.log.append(deepcopy(to_log))
            
            # should we stop ?
            if stop:
                break
            # if stop==True, then a collaboration occured, add 1 to counter
            n_iter += 1
            
            if n_iter == self.max_iter:
                break
            
    
    @njit
    def get_partitions_except_p(self, p):
        """
        Get all partition matrices except number p.
        This is used to get every remote partition matrix when p is the local data site.
        
        Parameters:
            p: int
                id of the partition matrix to ignore.
        """
        
        res = []
        for i, collab in self.collabs:
            if i != p:
                res.append(deepcopy(collabs[i].get_partition_matrix()))
        return res
    
    

TypeError: class members are not yet supported: __init__, launch_collab, get_partitions_except_p

# collaborator

In [58]:
class Collaborator:
    """
    An abstract class representing a collaborator.
    Collaborators can be of different types (running with different algorithms)
    e.g. Gtms, gaussian mixture models, other mixture models, or any kind of probabilistic model.
    """
    
    def __init__(self, X, Y=None, K=3, *args, **kwargs):
        """
        Parameters:
            X: pandas.DataFrame
                dataframe with the view of the collaborator.
            Y: pandas.DataFrame
                dataframe withh the labels.
            K: int
                number of clusters looked for.
        """
        self.X = X
        
    @abstractmethod
    def local_step(self):
        pass
    
    @abstractmethod
    def log_local(self):
        pass
    
    @abstractmethod
    def collaborate(self):
        pass
    
    @abstractmethod
    def log_collab(self):
        pass
    
    def get_partition_matrix(self):
        return self.R

In [ ]:
def Collaborator_gmm(Collaborator):
    """
    A Gaussian mixture model based collaborator
    """
    
    def __init__(self, X, Y=None, K=3, *args, **args):
        """
        Parameters:
        X: pd.DataFrame
            the features
        Y (optional): pd.DataFrame
            the labels
        K (optional): int
            the number of clusters
        """
        
        super().__init__(X=X, Y=Y, K=K)
        
    @njit
    def local_step(self):
        """
        
        """

# tests

In [14]:
class A:
    def __init__(self, a=0):
        self.a = a
        
    def double_a(self):
        try:
            return 2*self.a
        except:
            print("no attribute a")
        
class B(A):
    def __init__(self, a=0, b=0):
        self.a=a
        self.b=b

In [15]:
b = B()

In [16]:
b.double_a()

0